<a href="https://colab.research.google.com/github/Manoj2k27/testing/blob/main/ComputerNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##2.Write a program to implement the data link layer farming methods such as character stuffing and bit stuffing


In [ ]:
x = input("Enter the characters: ")

STX = 'stx'
ETX = 'etx'

segments = x.split()

framed_data = ""
for segment in segments:
    segment = segment.replace(STX, STX+STX).replace(ETX, ETX+ETX)
    framed_data += STX + segment + ETX + " "

framed_data = framed_data.strip()

print("Framed Data:", framed_data)


##bit stuffing


In [ ]:
x = input("Enter the bits: ")

STX = '01111110'
ETX = '01111110'  # corrected ETX to avoid confusion with STX
A = '111111'
B = '1111101'

segments = x.split(' ')

framed_data = ""
for segment in segments:
    segment = segment.replace(A, B)
    framed_data += STX + segment + ETX + " "

framed_data = framed_data.strip()

print("Framed Data:", framed_data)

#3. Write a program to implement data link layer framing method
#checkum

In [ ]:
def findChecksum(SentMessage, k):
    c1 = SentMessage[0:k]
    c2 = SentMessage[k:2*k]
    c3 = SentMessage[2*k:3*k]
    c4 = SentMessage[3*k:4*k]

    Sum = bin(int(c1, 2) + int(c2, 2) + int(c3, 2) + int(c4, 2))[2:]

    if len(Sum) > k:
        x = len(Sum) - k
        Sum = bin(int(Sum[0:x], 2) + int(Sum[x:], 2))[2:]
    if len(Sum) < k:
        Sum = '0' * (k - len(Sum)) + Sum

    Checksum = ''
    for i in Sum:
        if i == '1':
            Checksum += '0'
        else:
            Checksum += '1'
    return Checksum


def checkReceiverChecksum(ReceivedMessage, k):

    segments = [ReceivedMessage[i:i + k] for i in range(0, len(ReceivedMessage), k)]


    total_sum = sum(int(segment, 2) for segment in segments)


    ReceiverSum = bin(total_sum)[2:]


    if len(ReceiverSum) > k:
        x = len(ReceiverSum) - k
        ReceiverSum = bin(int(ReceiverSum[0:x], 2) + int(ReceiverSum[x:], 2))[2:]


    ReceiverChecksum = ''
    for i in ReceiverSum:
        if i == '1':
            ReceiverChecksum += '0'
        else:
            ReceiverChecksum += '1'


    if len(ReceiverChecksum) < k:
        ReceiverChecksum = '0' * (k - len(ReceiverChecksum)) + ReceiverChecksum

    return ReceiverChecksum


SentMessage = input("Enter sent message: ")
k = int(input("Enter frame size: "))

Checksum = findChecksum(SentMessage, k)
print("SENDER SIDE CHECKSUM: ", Checksum)


CombinedMessage = Checksum + SentMessage
print("SENDER MESSAGE: ", CombinedMessage)
receiverMessage=input("Enter receiver message:")

ReceiverChecksum = checkReceiverChecksum(receiverMessage, k)
print("RECEIVER SIDE CHECKSUM: ", ReceiverChecksum)
if ReceiverChecksum == '0' * k:
    print("Receiver Checksum is equal to 0. Therefore,")
    print("STATUS: ACCEPTED")
else:
    print("Receiver Checksum is not equal to 0. Therefore,")
    print("STATUS: ERROR DETECTED")

#4.Haming Code


In [ ]:
def calcRedundantBits(m):
    for i in range(m):
        if (2**i >= m + i + 1):
            return i

def posRedundantBits(data, r):
    j = 0
    k = 1
    m = len(data)
    res = ''

    for i in range(1, m + r + 1):
        if (i == 2**j):
            res = res + '0'
            j += 1
        else:
            res = res + data[-1 * k]
            k += 1

    return res[::-1]

def calcParityBits(arr, r, parity_type):
    n = len(arr)

    parity_bits = []
    for i in range(r):
        val = 0
        for j in range(1, n + 1):
            if (j & (2**i) == (2**i)):
                val = val ^ int(arr[-1 * j])

        # Adjust parity bit based on chosen parity type
        if parity_type == "odd":
            val = 1 if val == 0 else 0  # Flip the parity bit to ensure odd parity
        parity_bits.append(val)
        arr = arr[:n - (2**i)] + str(val) + arr[n - (2**i) + 1:]
    print("Number of parity bits:", len(parity_bits))
    print(f"Sender's parity bits: {parity_bits[::-1]}")  # Print sender's parity bits
    return arr

def detectError(arr, nr, parity_type):
    n = len(arr)
    res = 0
    parity_bits_receiver = []

    for i in range(nr):
        val = 0
        for j in range(1, n + 1):
            if (j & (2**i) == (2**i)):
                val = val ^ int(arr[-1 * j])

        parity_bits_receiver.append(val)
        res = res + val * (2**i)  # Convert binary parity error to position

    # Check parity type to ensure proper error detection
    if parity_type == "odd":
        for i in range(len(parity_bits_receiver)):
            if parity_bits_receiver[i] == 0:
                parity_bits_receiver[i] = 1  # Adjust to match odd parity
            else:
                parity_bits_receiver[i] = 0
    print("Number of parity bits:", len(parity_bits_receiver))
    print(f"Receiver's parity bits: {parity_bits_receiver[::-1]}")  # Print receiver's parity bits
    return res

# Main program
data = input("Enter the data to be transmitted (binary): ")
m = len(data)

# Get parity option from user (odd/even)
parity_type = input("Enter the parity type (odd/even): ").strip().lower()
while parity_type not in ['odd', 'even']:
    print("Invalid input! Please enter 'odd' or 'even'.")
    parity_type = input("Enter the parity type (odd/even): ").strip().lower()

r = calcRedundantBits(m)

arr = posRedundantBits(data, r)
arr = calcParityBits(arr, r, parity_type)

print("Code word is:", arr)

# Now instead of simulating an error, we will take the receiver's data as input
received_data = input("Enter the received data (binary): ")

# Check if lengths are equal
if len(received_data) != len(arr):
    print("Not equal length")
else:
    # Detect any errors in the received data
    correction = detectError(received_data, r, parity_type)

    if correction == 0:
        print("There is no error in the received message.")
    else:
        print("The position of error is", correction, "from the right (LSB).")

        # Correct the error by flipping the bit at the error position
        corrected_data = list(received_data)
        corrected_data[-correction] = '0' if corrected_data[-correction] == '1' else '1'
        corrected_data = ''.join(corrected_data)

        print("Corrected code word:", corrected_data)  # Print the corrected code word


#5.CRC12 CRC16

In [ ]:
def xor(a, b):
    """Perform XOR operation on two bits."""
    return 0 if a == b else 1

def calculate_crc(data, divisor):
    """
    Calculate CRC for the given data and divisor.
    """
    data_length = len(data)
    divisor_length = len(divisor)
    crc = data.copy()  # Copy the data to avoid modifying the original

    # Perform CRC division
    for i in range(data_length - divisor_length + 1):
        if crc[i] == 1:  # Only perform XOR if the first bit is 1
            for j in range(divisor_length):
                crc[i + j] = xor(crc[i + j], divisor[j])

    # Return the remainder (CRC)
    return crc[-(divisor_length - 1):]

def get_polynomial_representation(divisor):
    polynomial = []
    for i, bit in enumerate(divisor):
        if bit == 1:
            power = len(divisor) - i - 1
            if power == 0:
                polynomial.append("1")
            elif power == 1:
                polynomial.append("x")
            else:
                polynomial.append(f"x^{power}")
    return " + ".join(polynomial)

def main():
    # Input data and divisor
    try:
        # Input data as a single string of bits
        data_input = input("Enter the data bits (e.g., 10110): ")
        data = [int(bit) for bit in data_input]  # Convert string to list of integers

        # Input divisor as a single string of bits
        divisor_input = input("Enter the divisor bits (e.g., 1101): ")
        divisor = [int(bit) for bit in divisor_input]  # Convert string to list of integers

        # Display the generator polynomial
        polynomial = get_polynomial_representation(divisor)
        print(f"Generator polynomial: {polynomial}")

        # Append zeros to the data for CRC calculation
        data_with_zeros = data + [0] * (len(divisor) - 1)

        # Calculate CRC
        crc = calculate_crc(data_with_zeros, divisor)
        print("The CRC is:", "".join(map(str, crc)))

        # Prepare the data to be sent (original data + CRC)
        transmitted_data = data + crc
        print("The data to be sent is:", "".join(map(str, transmitted_data)))

        # Simulate receiver side
        received_input = input("Enter the received data bits (e.g., 10110010): ")
        received_data = [int(bit) for bit in received_input]  # Convert string to list of integers

        # Check for errors
        remainder = calculate_crc(received_data, divisor)
        print("The remainder is:", "".join(map(str, remainder)))

        if all(bit == 0 for bit in remainder):
            print("No error detected. Data is correct.")
        else:
            print("Error detected in the received data.")

    except ValueError:
        print("Invalid input. Please enter binary strings only (e.g., 10101).")

if __name__ == "__main__":
    main()

#6.Socket Programming


server

In [ ]:
#---------------------------sender----------------------------------
import socket

def start_server():
    try:
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        host = '127.0.0.1'
        port = 12345
        server_socket.bind((host, port))
        server_socket.listen(5)
        print(f"Server started at {host}:{port}. Waiting for connections...")
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address} has been established.")
        data = client_socket.recv(1024)
        if data:
            print(f"Received from client: {data.decode()}")
            client_socket.send(b"Message received!")
        print(f"Closing connection to {client_address}")
        client_socket.close()

    except Exception as e:
        print(f"An error occurred with the server: {e}")
    finally:
        server_socket.close()

if __name__ == '__main__':
    start_server()


receiver


In [ ]:
#----------------------receiver-----------
import socket

def start_client():
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        host = '127.0.0.1'
        port = 12345
        client_socket.connect((host, port))
        print("Connected to server. Type your message:")

        message = input("Enter message: ")
        client_socket.send(message.encode())
        data = client_socket.recv(1024)
        print(f"Server response: {data.decode()}")

    except ConnectionRefusedError:
        print("Could not connect to the server. Ensure the server is running.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        client_socket.close()

if __name__ == '__main__':
    start_client()

# 7. Stop and Wait Arq

#.Client

In [ ]:
import socket
import time

def start_client(message, timeout):
    host = 'localhost'
    port = 12678
    sequence_number = 0

    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        print(f"Connecting to server at {host}:{port}...")
        client_socket.connect((host, port))
        print(f"Connected to server at {host}:{port}")

        frames = message.split(' ')  # Divide message into frames based on spaces

        for frame in frames:
            while True:
                data = f"{sequence_number}:{frame}"
                print(f"Sending frame: {data}")
                client_socket.sendall(data.encode())  # Just send without flush

                client_socket.settimeout(timeout)
                try:
                    ack = client_socket.recv(1024).decode()
                    if ack == f"ACK:{sequence_number}":
                        print(f"Received acknowledgment: {ack}")
                        sequence_number = 1 - sequence_number  # Flip sequence number (0 -> 1, 1 -> 0)
                        break
                except socket.timeout:
                    print("Timeout occurred, resending frame...")

    except socket.error as e:
        print(f"Socket error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        client_socket.close()

if __name__ == "__main__":
    message = input("Enter the message to send to the server: ")
    timeout = float(input("Enter the timeout duration in seconds: "))
    start_client(message, timeout)


server

In [ ]:

import socket

def start_server():
    host = 'localhost'
    port = 12345
    expected_sequence_number = 0
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Reuse the address

    server_socket.bind((host, port))
    server_socket.listen(1)
    print(f"Server listening on {host}:{port}...")
    client_socket, client_address = server_socket.accept()
    print(f"Connection established with {client_address}")
    while True:
        data = client_socket.recv(1024).decode()
        if not data:
            print("No data received, closing connection.")
            break

        if data.count(':') == 1:
            sequence_number, frame = data.split(':')
            sequence_number = int(sequence_number)

            if sequence_number == expected_sequence_number:
                print(f"Received from client: {frame}")
                user_input = input("Acknowledge the frame? (yes/no): ")
                if user_input.lower() == 'yes':
                    ack = f"ACK:{sequence_number}"
                    client_socket.sendall(ack.encode())
                    print(f"Sent acknowledgment: {ack}")
                    expected_sequence_number = 1 - expected_sequence_number  # Flip sequence number (0 -> 1, 1 -> 0)
                else:
                    print("Acknowledgment not sent, simulating lost ACK.")
            else:
                print(f"Received out-of-order frame with sequence number: {sequence_number}")
        else:
            print(f"Invalid data received: {data}. Ignoring.")

    client_socket.close()
    server_socket.close()

if __name__ == "__main__":
    start_server()



#8.Sliding Window Program

sender

In [ ]:
import socket
import time

def sender(total_packets, window_size, host='localhost', port=12346):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    client_socket.settimeout(2)  # Set timeout to avoid infinite blocking

    base = 1  # First packet in the window
    next_seq_num = 1  # Next packet to send
    acked_packets = set()  # Store acknowledged packets

    print(" Sender is starting...\n")

    while base <= total_packets:
        # Send packets within the window
        while next_seq_num < base + window_size and next_seq_num <= total_packets:
            if next_seq_num not in acked_packets:  # Send only unacknowledged packets
                packet = f"Packet_{next_seq_num}"
                client_socket.sendto(packet.encode(), (host, port))
                print(f" Sent: {packet}")
            next_seq_num += 1

        try:
            # Receive ACKs
            ack, _ = client_socket.recvfrom(1024)
            ack_num = int(ack.decode().split("_")[1])  # Extract packet number from "ACK_X"
            print(f" Received Acknowledgment: {ack.decode()}")

            # Mark packet as acknowledged
            acked_packets.add(ack_num)

            # Slide the window if base is acknowledged
            while base in acked_packets:
                base += 1  # Move window forward

        except socket.timeout:
            print(" Timeout: No ACK received, resending unacknowledged packets...\n")
            next_seq_num = base  # Reset to base to resend unacknowledged packets

        time.sleep(1)  # Simulating network delay

    print(" All packets sent and acknowledged successfully.")
    client_socket.close()

if __name__ == "__main__":
    total_packets = int(input("Enter total number of packets: "))
    window_size = int(input("Enter window size: "))
    sender(total_packets, window_size)

receiver

In [ ]:
#-----------------------------------receiver--------------------


import socket
import time

def receiver(host='localhost', port=12346):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    # Allow address reuse to prevent 'address already in use' errors
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    server_socket.bind((host, port))
    print("Receiver is ready...\n")

    expected_packet = 1
    received_packets = set()

    while True:
        try:
            packet, client_address = server_socket.recvfrom(1024)
            packet_num = int(packet.decode().split("_")[1])
            print(f" Received: {packet.decode()}")

            if packet_num == expected_packet:
                received_packets.add(packet_num)
                print(f" Packet {packet_num} received in order.")

                ack_message = f"ACK_{packet_num}"
                server_socket.sendto(ack_message.encode(), client_address)
                print(f" Sent Acknowledgment: {ack_message}\n")

                expected_packet += 1

            elif packet_num in received_packets:
                print(f" Duplicate packet {packet_num} received, ignoring.")

            time.sleep(1)

        except Exception as e:
            print(f"⚠ Error: {e}")
            break

    server_socket.close()

if __name__ == "__main__":
    receiver()


#9.GO-BACK-N ARQ


receiver

In [ ]:
# receiver.py (Go-Back-N)
import socket
import random

def receiver():
    receiver_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    receiver_address = ('127.0.0.1', 12345)
    receiver_socket.bind(receiver_address)

    expected_seq_num = 0

    while True:
        try:
            receiver_socket.settimeout(10)
            frame, sender_address = receiver_socket.recvfrom(1024)
            frame_data = frame.decode()
            frame_seq = int(frame_data.split(" ")[1])

            if random.random() < 0.2: #simulate packet loss
                print(f"Packet loss: {frame_data}")
                continue

            if frame_seq == expected_seq_num:
                print(f"Received {frame_data}")
                receiver_socket.sendto(str(frame_seq).encode(), sender_address)
                print(f"Sending ACK {frame_seq}")
                expected_seq_num += 1
            elif frame_seq < expected_seq_num: # send duplicate ack for previously received frame
                receiver_socket.sendto(str(frame_seq).encode(), sender_address)
                print(f"Sending duplicate ACK {frame_seq}")
            else:
                print(f"Out-of-order frame: {frame_data}. Expected: {expected_seq_num}, Received: {frame_seq}")

        except socket.timeout:
            print("Receiver timeout. Exiting.")
            break

    receiver_socket.close()

if __name__ == "__main__":
    receiver()


sender


In [ ]:
# sender.py (Go-Back-N, dynamic ack_received)
import socket
import time

def sender():
    sender_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    receiver_address = ('127.0.0.1', 12345)

    num_frames = int(input("Enter the number of frames: "))
    window_size = int(input("Enter the window size: "))

    frames = [f"Frame {i}" for i in range(num_frames)]
    send_base = 0
    next_seq_num = 0
    ack_received = {}  # Use a dictionary instead of a list
    timeout_occurred = False
    timeout_start = 0

    while send_base < num_frames:
        while next_seq_num < min(send_base + window_size, num_frames):
            if next_seq_num not in ack_received or not ack_received[next_seq_num]:
                print(f"Sending {frames[next_seq_num]} (Seq: {next_seq_num})")
                sender_socket.sendto(frames[next_seq_num].encode(), receiver_address)
                if send_base == next_seq_num and not timeout_occurred:
                    timeout_start = time.time()
                    timeout_occurred = True
                time.sleep(0.1)
            next_seq_num += 1

        try:
            sender_socket.settimeout(0.1)
            ack_message, _ = sender_socket.recvfrom(1024)
            ack_seq = int(ack_message.decode())
            print(f"Received ACK for Seq: {ack_seq}")
            ack_received[ack_seq] = True

            if ack_seq == send_base:
                while send_base < num_frames and send_base in ack_received and ack_received[send_base]:
                    send_base += 1
                if send_base == next_seq_num:
                    timeout_occurred = False

        except socket.timeout:
            if timeout_occurred and time.time() - timeout_start > 0.5:
                print("Timeout: Resending from base.")
                next_seq_num = send_base
                timeout_start = time.time()

    print("All frames sent and acknowledged.")
    sender_socket.close()


if __name__ == "__main__":
    sender()

#10. SELECTIVE REPEAT ARQ


In [ ]:
import socket
import time

def sender():
    sender_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    receiver_address = ('127.0.0.1', 12345)

    num_frames = int(input("Enter the number of frames: "))
    window_size = int(input("Enter the window size: "))

    frames = [f"Frame {i}" for i in range(num_frames)]
    send_base = 0
    next_seq_num = 0
    ack_received = [False] * num_frames
    frame_sent_time = {}  # Dictionary to store sent timestamps
    timeout_interval = 0.5  # Timeout interval in seconds

    while send_base < num_frames:
        # Send frames within the window
        while next_seq_num < min(send_base + window_size, num_frames):
            if not ack_received[next_seq_num]:
                print(f"Sending {frames[next_seq_num]} (Seq: {next_seq_num})")
                sender_socket.sendto(frames[next_seq_num].encode(), receiver_address)
                frame_sent_time[next_seq_num] = time.time()  # Store sent time
            next_seq_num += 1

        # Check for acknowledgments and retransmission
        try:
            sender_socket.settimeout(0.1)  # Short timeout to check for ACKs
            ack_message, _ = sender_socket.recvfrom(1024)
            ack_seq = int(ack_message.decode())
            print(f"Received ACK for Seq: {ack_seq}")
            ack_received[ack_seq] = True

            # Slide the window
            while send_base < num_frames and ack_received[send_base]:
                send_base += 1

        except socket.timeout:
            # Check for timeouts for individual frames
            current_time = time.time()
            for i in range(send_base, min(send_base + window_size, num_frames)):
                if not ack_received[i] and (current_time - frame_sent_time.get(i, 0)) > timeout_interval:
                    print(f"Timeout: Resending {frames[i]} (Seq: {i})")
                    sender_socket.sendto(frames[i].encode(), receiver_address)
                    frame_sent_time[i] = current_time  # Update sent time

    print("All frames sent and acknowledged.")
    sender_socket.close()

if __name__ == "__main__":
    sender()


In [ ]:
import socket
import random

def receiver():
    receiver_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    receiver_address = ('127.0.0.1', 12345)
    receiver_socket.bind(receiver_address)

    expected_seq_num = 0
    received_frames = {}  # Buffer for out-of-order frames

    while True:
        try:
            receiver_socket.settimeout(5)  # Timeout for waiting frames
            frame, sender_address = receiver_socket.recvfrom(1024)
            frame_data = frame.decode()
            frame_seq = int(frame_data.split(" ")[1])

            if random.random() < 0.2:  # Simulate packet loss
                print(f"Packet loss: {frame_data}")
                continue

            if frame_seq == expected_seq_num:
                print(f"Received {frame_data}")
                received_frames[frame_seq] = frame_data  # Store the frame
                receiver_socket.sendto(str(frame_seq).encode(), sender_address)
                print(f"Sending ACK {frame_seq}")
                expected_seq_num += 1

                # Deliver buffered frames in order
                while expected_seq_num in received_frames:
                    print(f"Delivered {received_frames[expected_seq_num]}")
                    receiver_socket.sendto(str(expected_seq_num).encode(), sender_address)
                    print(f"Sending ACK {expected_seq_num}")
                    expected_seq_num += 1

            elif frame_seq > expected_seq_num:  # Out-of-order frame
                received_frames[frame_seq] = frame_data
                receiver_socket.sendto(str(frame_seq).encode(), sender_address)
                print(f"Received out-of-order {frame_data}, buffering. Sending ACK {frame_seq}")

            elif frame_seq < expected_seq_num:  # Duplicate frame
                receiver_socket.sendto(str(frame_seq).encode(), sender_address)
                print(f"Duplicate frame {frame_data}, Sending duplicate ACK {frame_seq}")

        except socket.timeout:
            print("Receiver timeout. Exiting.")
            break

    receiver_socket.close()

if __name__ == "__main__":
    receiver()

#DJAKSTRAS ALGORITHM
